<a href="https://colab.research.google.com/github/weedge/doraemon-nb/blob/main/GLM_4_Voice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi && lscpu

# GLM-4-Voice

In [1]:
!git clone --recurse-submodules https://github.com/weedge/GLM-4-Voice.git


Cloning into 'GLM-4-Voice'...
remote: Enumerating objects: 210, done.
remote: Counting objects: 100% (66/66), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 210 (delta 56), reused 46 (delta 45), pack-reused 144 (from 1)
Receiving objects: 100% (210/210), 506.09 KiB | 25.30 MiB/s, done.
Resolving deltas: 100% (90/90), done.
Submodule 'third_party/Matcha-TTS' (https://github.com/shivammehta25/Matcha-TTS) registered for path 'third_party/Matcha-TTS'
Cloning into '/content/GLM-4-Voice/third_party/Matcha-TTS'...
remote: Enumerating objects: 921, done.        
remote: Counting objects: 100% (406/406), done.        
remote: Compressing objects: 100% (124/124), done.        
remote: Total 921 (delta 336), reused 283 (delta 282), pack-reused 515 (from 1)        
Receiving objects: 100% (921/921), 64.08 MiB | 16.93 MiB/s, done.
Resolving deltas: 100% (421/421), done.
Submodule path 'third_party/Matcha-TTS': checked out 'dd9105b34bf2be2230f4aa1e4769fb586a3c824e'


In [2]:
%cd /content/GLM-4-Voice


/content/GLM-4-Voice


In [ ]:
!cat requirements.txt

In [ ]:
!pip install -q -r requirements.txt


In [5]:
from google.colab import userdata
NGROK_TOKEN=userdata.get('NGROK_TOKEN')
HF_TOKEN=userdata.get('HF_TOKEN')


# download model

In [6]:
%cd /content/GLM-4-Voice

/content/GLM-4-Voice


In [ ]:
!huggingface-cli login --token $HF_TOKEN --add-to-git-credential

In [ ]:
# GLM-4-Voice-Tokenizer: Trained by adding vector quantization to the encoder part of Whisper,
# converting continuous speech input into discrete tokens.
# Each second of audio is converted into 12.5 discrete tokens.

!huggingface-cli download THUDM/glm-4-voice-tokenizer --local-dir THUDM/glm-4-voice-tokenizer --local-dir-use-symlinks False

In [ ]:
# GLM-4-Voice-9B: Pre-trained and aligned on speech modality based on GLM-4-9B,
# enabling understanding and generation of discretized speech.

!huggingface-cli download THUDM/glm-4-voice-9b --local-dir THUDM/glm-4-voice-9b --local-dir-use-symlinks False

In [ ]:
# GLM-4-Voice-Decoder: A speech decoder supporting streaming inference, retrained based on CosyVoice,
# converting discrete speech tokens into continuous speech output.
# Generation can start with as few as 10 audio tokens, reducing conversation latency.

!git lfs install
!git clone https://huggingface.co/THUDM/glm-4-voice-decoder /content/GLM-4-Voice/THUDM/glm-4-voice-decoder

# run server and gradio web ui

In [11]:
%cd /content/GLM-4-Voice

/content/GLM-4-Voice


In [ ]:
# need run on A100 GPU
# u can run server on terminal
!python model_server.py --host localhost --model-path THUDM/glm-4-voice-9b --port 10000 --dtype bfloat16 --device cuda:0

In [ ]:
# run glm-4-voice-9b with int4 on T4 GPU
# u can run server on terminal
!python model_server.py --host 0.0.0.0 --model-path THUDM/glm-4-voice-9b --port 10000 --dtype int4 --device cuda:0

In [ ]:
!ngrok config add-authtoken $NGROK_TOKEN

In [ ]:
# run glm-4-voice-9b with int4 on T4 GPU with ngrok proxy
!python model_server.py --host 0.0.0.0 --model-path THUDM/glm-4-voice-9b --port 10000 --dtype int4 --device cuda:0 --ngrok 1

In [ ]:
# open ngrok proxy url to run web gradio ui
!python web_demo.py --tokenizer-path  THUDM/glm-4-voice-tokenizer --model-path THUDM/glm-4-voice-9b --flow-path THUDM/glm-4-voice-decoder --ngrok 1

Public URL: https://2c99-34-126-133-36.ngrok-free.app
/usr/local/lib/python3.10/dist-packages/torch/_jit_internal.py:739: FutureWarning: ignore(True) has been deprecated. TorchScript will now drop the function call on compilation. Use torch.jit.unused now. {}
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/diffusers/models/lora.py:393: FutureWarning: `LoRACompatibleLinear` is deprecated and will be removed in version 1.0.0. Use of `LoRACompatibleLinear` is deprecated. Please switch to PEFT backend by installing PEFT: `pip install peft`.
  deprecate("LoRACompatibleLinear", "1.0.0", deprecation_message)
/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
* Running on local URL:  http://0.0.0.0:8888

To create a public link, set `shar